```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Linear Coriolis term

The coriolis parameter is modeled as a linear function
$$f = \tilde{f} + \beta y$$
where $y=0$ currently represent the lower boundary of the first non-ghost-cell of the domain.

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_linearCoriolis_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

## Parameters

Kai and Göran: Go nuts and make changes here :)

In [ ]:
nx = 100
ny = 200

dx = 200.0
dy = 200.0

# The simulation have the following structure:
# for 1:T
#     simulate until t = t0 + sub_t 
#     add plot to animation

#T = 200
#sub_t = 300.0

T = 100
sub_t = 20.0

dt = 0.5


g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 1

# Coriolis parameters: f + beta * y
f = 0.02
beta = 0.0002/dy

wind = Common.WindStressParams(type=99) # type = 99 corresponds to no wind.

# Boundary condions
# 1: reflective wall, 
# 2: periodic, 
# 3: open (flow relaxation scheme), 
bcSettings = 3


waterHeight = 60

#Calculate radius from center for plotting
x_center = dx*nx*0.5
y_center = dy*ny*0.5
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

reload(Common)

### Creating velocity field from arbitrary initial $\eta$

Makes approximations based on central differences for
$$ hv = \frac{gh}{f} \frac{\partial \eta}{\partial x} $$
and
$$ hu = \frac{gh}{f} \frac{\partial \eta}{\partial y} $$


In [ ]:
def initializeBalancedVelocityFieldStaggered(eta, H, hu, hv, f, beta, g, nx, ny, dx, dy, ghosts):
    for j in range(1, ny+ghosts[0]+ghosts[2]-1):
        coriolis = f + beta*j*dy
        for i in range(1, nx+ghosts[1]+ghosts[3]-1):
            eta_pluss = (eta[j+1, i] + eta[j+1, i-1])/2.0
            eta_minus = (eta[j-1, i] + eta[j-1, i-1])/2.0
            h_mid = (eta[j,i] + H[j,i] + eta[j, i-1] + H[j,i-1])/2.0
            hu[j,i] = -(g/coriolis)*h_mid*(eta_pluss - eta_minus)/(2.0*dy)
    for j in range(1, ny+ghosts[0]+ghosts[2]-1):
        coriolis = f + beta*j*dy
        for i in range(1, nx+ghosts[1]+ghosts[3]-1):
            eta_pluss = (eta[j, i+1] + eta[j-1, i+1])/2.0
            eta_minus = (eta[j, i-1] + eta[j-1, i-1])/2.0
            h_mid = (eta[j,i] + H[j,i] + eta[j-1,i] + H[j-1,i])/2.0
            hv[j,i] =  (g/coriolis)*h_mid*(eta_pluss - eta_minus)/(2.0*dx)

## Centered in time, centered in space

In [ ]:
#Centered in time, centered in space

#Clean up old simulator if any:
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()

ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
#addCentralBump(eta0, nx, ny, dx, dy, ghosts)
addWideDebugBump(eta0, nx, ny, dx, dy, 0.5, 0.5, 20, ghosts)
initializeBalancedVelocityFieldStaggered(eta0, h0, u0, v0, f, beta, g, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(CTCS)
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     h0, eta0, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, A, \
                     coriolis_beta=beta, \
                     wind_stress=wind, \
                     write_netcdf=False, \
                     boundary_conditions=boundaryConditions )


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] , 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                                v0[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = ctcs_sim.step(sub_t)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1.0
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                 brighten*v1[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) +  "\tMax h: " + str(np.max(eta1 + 60)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim